In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from Agents.DQNAgent import DQNAgent
from Boards.Speed_leedo_2p import FullBoard
import sys
import tqdm
import warnings
import os
import pickle
from multiprocessing import Pool
from tornado import concurrent
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

env = FullBoard()

# 0 - load data, 1 - read/process, 2 - do nothing
main_data = 2
game_history_data = 2
state_process = 0
data_q_val = 1

post = 'sep_6_to_sep_10'

2022-09-23 21:01:28.548154: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
if main_data != 2:
    if main_data:
        query="""
            select *
            from `analytics-156605.barath.ludo_details_1`
        """

        df = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

        df.to_feather('../data/ludo_2p_data_' + post)

    else:
        
        df = pd.read_feather('../data/ludo_2p_data_' + post)

    df = df.to_dict('records')

In [3]:
# if game_history_data != 2:
#     if game_history_data:

#         query="""
#                 select *
#                 from `analytics-156605.barath.game_history`
#             """

#         game_history = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

#         game_history.to_feather('../data/game_history_1_to_15.feather')
        
#     else:

#         game_history = pd.read_feather('../data/game_history_1_to_15.feather')

#     game_history = game_history.to_dict('records')
#     game_history = {item['user_id']:item['f0_'] for item in game_history if item is not None}
#     # game_history = game_history.groupby('user_id').apply(lambda x: dict(zip(x.game_ref_id, x.last_5))).to_dict()

In [4]:
if state_process != 2:
    if state_process:

        warnings.filterwarnings('ignore')
        
        def func(group):

            def extract_pos(x):
                return np.array([json.loads(i) for i in x])

            def extract_home(x):
                temp = np.array([json.loads(i) for i in x])
                temp[temp=='HOME'] = 1
                temp[temp!='HOME'] = 0
                temp = temp.astype('float32')
                temp = np.sum(temp, -1)
                return np.expand_dims(temp, -1)

            def my_argmax(a):
                rows = np.where(a == a.max(axis=1)[:, None])[0]
                rows_multiple_max = rows[:-1][rows[:-1] == rows[1:]]
                my_argmax = a.argmax(axis=1)
                my_argmax[rows_multiple_max] = -1
                return my_argmax

            try:

                if len(group['pawns_position_1']) > 25 and len(group['pawns_position_2']) > 25:
                    
                    pawn_position_1 = extract_pos(group['pawns_position_1'])
                    pawn_position_2 = extract_pos(group['pawns_position_2'])
                    home_1 = extract_home(group['home_1'])
                    home_2 = extract_home(group['home_2'])

                    if len(pawn_position_1) != len(pawn_position_2):
                        length = np.min([len(pawn_position_1), len(pawn_position_2)])
                        pawn_position_1 = pawn_position_1[:length]
                        pawn_position_2 = pawn_position_2[:length]
                        home_1 = home_1[:length]
                        home_2 = home_2[:length]

                    first_user = group['from_user'][0]
                    second_user = group['opponent'][0]
                    
                    from_user = np.array(group['from_user'])
                    dice = np.array(group['dice_score'])

                    user_ids = (from_user == second_user).astype('float32')
                    user_ids = user_ids[1:]
                    dice = dice[1:]

                    pos = np.concatenate((pawn_position_1, pawn_position_2), axis=-1)
                    home = np.concatenate((home_1, home_2), -1)
                    score = [np.sum(json.loads(group['score_1'][-1])), np.sum(json.loads(group['score_2'][-1]))]
                    mu_0 = group['mu'][0]
                    sigma_0 = group['sigma'][0]
                    idx = np.where(group['from_user'] == second_user)[0][0]
                    mu_1 = group['mu'][idx]
                    sigma_1 = group['sigma'][idx]

                    score = np.array(score)

                    action_1 = my_argmax((pawn_position_1[1:] - pawn_position_1[:-1]))
                    action_2 = my_argmax((pawn_position_2[1:] - pawn_position_2[:-1]))
                    action = np.copy(action_1)
                    action[action==-1] = action_2[action==-1]

                    states = np.concatenate((np.expand_dims(dice, -1).astype('float32'), pos[:-1,:], home[:-1,:]), -1)

                    return {'division': group['division'], 'user_id_0': first_user, 'user_id_1': second_user,
                                'user_ids': user_ids, 'score': score, 
                                'states': states, 'action': action,
                                'mu_0': mu_0, 'sigma_0': sigma_0,
                                'mu_1': mu_1, 'sigma_1': sigma_1}
            except:
                return None

        p = Pool(16)

        results = p.map(func, tqdm.tqdm(df))

        results = [i for i in results if i is not None]
        
        with open('../data/state_data_' + post + '.pkl', 'wb') as f:
            pickle.dump(results, f)

    else:
        
        with open('../data/state_data_' + post + '.pkl', 'rb') as f:
            results = pickle.load(f)

100%|██████████| 2547029/2547029 [15:08<00:00, 2803.60it/s]


In [5]:
if data_q_val:
    
    agent0 = DQNAgent(env.state_size(), env.action_size(), env.max_val())
    agent0.load('./model_output/DQN_2p_v2/0004/weights_100000.hdf5')

    # for dat in tqdm.tqdm(data):
        
        # group = data[dat]

    def func1(group):

        if len(np.squeeze(np.array(group['states'])).shape) == 2:

            q_vals = np.squeeze(agent0.model.predict(np.reshape(group['states'], [len(group['states']),11,1])/69))
            q_temp = np.array([q_vals[i,group['action'][i]] for i in range(len(group['states']))])
            q_max = np.max(q_vals, -1)
            q_min = np.min(q_vals, -1)
            # data[dat].update({'q_user_0': q_temp[group['user_ids']==0],
            #                   'q_user_1': q_temp[group['user_ids']==1],
            #                   'q_max_user_0': q_max[group['user_ids']==0],
            #                   'q_max_user_1': q_max[group['user_ids']==1]})
            
            return {'q_user_0': q_temp[group['user_ids']==0],
                    'q_user_1': q_temp[group['user_ids']==1],
                    'q_max_user_0': q_max[group['user_ids']==0],
                    'q_max_user_1': q_max[group['user_ids']==1],
                    'q_min_user_0': q_min[group['user_ids']==0],
                    'q_min_user_1': q_min[group['user_ids']==1]}

    # futures = []
    # with concurrent.futures.ThreadPoolExecutor(max_workers=24) as executor:
    #     for d in tqdm.tqdm(results):
    #         future = executor.submit(func1, d)
    #         futures.append(future)

    results1 = []
    for i, d in enumerate(tqdm.tqdm(results)):
        results1.append(func1(d))

        if i % 100000 == 0:
            with open('../data/q_val_data_' + post + '.pkl', 'wb') as f:
                pickle.dump([results, results1], f)

    # for future in tqdm.tqdm(futures):
    #     results1.append(future.result())

    with open('../data/q_val_data_' + post + '.pkl', 'wb') as f:
        pickle.dump([results, results1], f)

else:

    with open('../data/q_val_data_' + post + '.pkl', 'rb') as f:
        results, results1 = pickle.load(f)

2022-09-21 06:36:47.108462: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-21 06:36:47.109503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-21 06:36:47.144890: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-21 06:36:47.144940: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ds-rnd-t4-32c-02
2022-09-21 06:36:47.144948: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ds-rnd-t4-32c-02
2022-09-21 06:36:47.145071: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 465.19.1
2022-09-21 06:36:47.145099: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 465.19.1
2022-09-21 06:36:47.145105: I tensorflow/stre

KeyboardInterrupt: 

In [ ]:
# for i in range(len(results)):
#     results[i].update(results1[i])

# data = {item['division']:item for item in results if item is not None}

In [ ]:
# q_data = {}

# for dat in data:
#     if 'q_user_0' in data[dat]:
#         q_data[dat] = {}
#         q_data[dat][data[dat]['user_id_0']] = {'q': data[dat]['q_user_0'], 'q_max': data[dat]['q_max_user_0'], 
#                                             'win': int(data[dat]['score'][0] > data[dat]['score'][1]),
#                                             'mu': data[dat]['mu_0'], 'sigma': data[dat]['sigma_0']}
#         q_data[dat][data[dat]['user_id_1']] = {'q': data[dat]['q_user_1'], 'q_max': data[dat]['q_max_user_1'], 
#                                             'win': int(data[dat]['score'][1] > data[dat]['score'][0]),
#                                             'mu': data[dat]['mu_1'], 'sigma': data[dat]['sigma_1']}

In [ ]:
# feats = []
# y = []

# for dat in tqdm.tqdm(q_data):
#     user_id_0 = list(q_data[dat].keys())[0]
#     user_id_1 = list(q_data[dat].keys())[1]
#     if user_id_0 in game_history and user_id_1 in game_history and dat in game_history[user_id_0] and dat in game_history[user_id_1]:
#         pos_1 = np.where(game_history[user_id_0] == dat)[0][0]
#         pos_2 = np.where(game_history[user_id_1] == dat)[0][0]
#         if pos_1 >= 5 and pos_2 >= 5:
#             idx_1 = []
#             for i in range (0, pos_1):
#                 if game_history[user_id_0][i] in q_data:
#                     idx_1.append(True)
#                 else:
#                     idx_1.append(False)
#             idx_2 = []
#             for i in range (0, pos_2):
#                 if game_history[user_id_1][i] in q_data:
#                     idx_2.append(True)
#                 else:
#                     idx_2.append(False)
#             if np.sum(idx_1) >= 5 and np.sum(idx_2) >= 5:
#                 q_mean_0 = []
#                 q_max_mean_0 = []
#                 q_mean_1 = []
#                 q_max_mean_1 = []

#                 games = game_history[user_id_0][:pos_1][idx_1]
#                 games = games[-5:]
#                 for i in range(len(games)):
#                     q_mean_0.append(np.mean(q_data[games[i]][user_id_0]['q']))
#                     q_max_mean_0.append(np.mean(q_data[games[i]][user_id_0]['q_max']))

#                 games = game_history[user_id_1][:pos_2][idx_2]
#                 games = games[-5:]
#                 for i in range(len(games)):
#                     q_mean_1.append(np.mean(q_data[games[i]][user_id_1]['q']))
#                     q_max_mean_1.append(np.mean(q_data[games[i]][user_id_1]['q_max']))

#                 mu_0 = q_data[dat][user_id_0]['mu']
#                 mu_1 = q_data[dat][user_id_1]['mu']

#                 sigma_0 = q_data[dat][user_id_0]['sigma']
#                 sigma_1 = q_data[dat][user_id_1]['sigma']

#                 temp_feat = q_mean_0 + q_max_mean_0 + [mu_0, sigma_0] + q_mean_1 + q_max_mean_1 + [mu_1, sigma_1]
#                 temp_y = q_data[dat][user_id_0]['win']

#                 feats.append(temp_feat)
#                 y.append(temp_y)

In [ ]:
# with open('../data/ludo_qval_features_1_to_15.pkl', 'wb') as f:
#     pickle.dump([np.array(feats), np.array(y)], f)